<a href="https://colab.research.google.com/github/babupallam/Msc_AI_Module2_Natural_Language_Processing/blob/main/L10-Graph%20Convolutional%20Networks/Supporting_Documents/library_networkx/library_networkx_note__04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphSAGE and Graph Isomorphism Network (GIN), which are designed for scalable and expressive graph learning tasks.

# Demonstration 61: GraphSAGE Layer (Sample and Aggregate)


### GraphSAGE (Graph Sample and Aggregate)

GraphSAGE (Graph Sample and Aggregate) is a framework designed to generate node embeddings in a scalable way. Unlike many traditional graph neural networks that require all nodes' features and structure for each batch, GraphSAGE works by **sampling a fixed number of neighbors** for each node, which makes it more suitable for large graphs.

#### Key Concepts

1. **Sampling-Based Aggregation**:
   - Instead of aggregating all neighbors, GraphSAGE samples a fixed number of neighbors for each node.
   - This makes GraphSAGE more scalable, as it doesn’t require loading the entire graph at once.

2. **Neighbor Aggregation**:
   - GraphSAGE uses different types of aggregation functions, such as:
     - **Mean Aggregation**: Takes the average of the features of the sampled neighbors.
     - **LSTM Aggregation**: Uses an LSTM network to aggregate neighbor features, which can capture more complex patterns.
     - **Pooling Aggregation**: Applies a neural network layer followed by a pooling function (like max or mean pooling) to summarize neighbor features.

3. **Inductive Learning**:
   - GraphSAGE is designed for inductive tasks, where it can generalize to unseen nodes or graphs by learning functions for sampling and aggregating neighbors.

#### Workflow of GraphSAGE

1. **Sampling**: For each node, sample a fixed number of neighbors at each layer.
2. **Aggregation**: Use an aggregation function to combine the features of the sampled neighbors.
3. **Update**: Concatenate the node’s current representation with the aggregated features, then apply a neural network layer to produce the new node embedding.
4. **Stacking Layers**: Repeat the process for a fixed number of layers, effectively allowing each node to capture information from neighbors up to `k` hops away.

#### Applications of GraphSAGE

- **Social Networks**: Friend recommendation by embedding users based on their connections.
- **Biological Networks**: Predicting protein-protein interactions or classifying proteins based on neighborhood structures.
- **Knowledge Graphs**: Learning embeddings for entities that capture relational information.

### Graph Isomorphism Network (GIN)

The Graph Isomorphism Network (GIN) is a powerful type of graph neural network designed to **capture structural information** in a way that is as powerful as the Weisfeiler-Lehman (WL) graph isomorphism test. This is achieved by making GIN highly expressive, enabling it to distinguish between different graph structures more effectively than other GNNs.

#### Key Concepts

1. **Expressive Power**:
   - GIN is designed to be as expressive as the WL graph isomorphism test, which is a method for determining if two graphs are structurally identical.
   - GIN achieves this by using a unique aggregation and update function that captures subtle structural differences.

2. **Sum Aggregation**:
   - Unlike other GNNs that use mean or max pooling, GIN uses **sum aggregation** to combine features from neighbors.
   - Sum aggregation ensures that the network does not lose information by averaging out differences, which can be crucial for distinguishing nodes with similar neighborhoods.

3. **MLP Update**:
   - After aggregation, GIN applies a **multilayer perceptron (MLP)** to the combined features, which gives it more flexibility to learn complex transformations.
   - The MLP updates the node’s representation based on both its own features and the aggregated neighbor features.

4. **Injective Aggregation**:
   - The design of GIN makes it possible to represent different neighborhoods uniquely, which helps it distinguish between nodes that might look identical to other GNNs.

#### Workflow of GIN

1. **Sum Aggregation**: Aggregate neighbor features using sum rather than mean or max.
2. **Update with MLP**: Apply an MLP to the aggregated features plus the node’s own features.
3. **Stacking Layers**: Stack multiple GIN layers to allow each node to capture information from multiple hops.

#### Applications of GIN

- **Molecular Graphs**: Useful in predicting molecular properties, as it captures structural information in molecules very effectively.
- **Graph Classification**: Suitable for tasks where the goal is to classify entire graphs rather than individual nodes, such as classifying chemical compounds or proteins.
- **Structural Pattern Detection**: GIN’s expressive power makes it ideal for detecting structural patterns in complex graphs.

### Comparison of GraphSAGE and GIN

| Aspect                     | GraphSAGE                                           | GIN                                      |
|----------------------------|-----------------------------------------------------|------------------------------------------|
| **Aggregation Method**     | Mean, LSTM, Pooling (sampling-based)                | Sum                                      |
| **Expressive Power**       | Less expressive, designed for scalability           | Highly expressive, can capture graph isomorphisms |
| **Scalability**            | High (samples neighbors, avoids entire graph loading) | Lower scalability (needs full neighborhood aggregation) |
| **Inductive Learning**     | Yes (can generalize to unseen nodes)                | No (trained on a fixed graph)            |
| **Typical Use Cases**      | Large, dynamic graphs (social networks, knowledge graphs) | Small, structured graphs (molecular graphs, chemistry) |
| **Update Mechanism**       | Aggregation followed by concatenation               | Sum aggregation followed by MLP update   |

### Summary

- **GraphSAGE** is best suited for large, dynamic graphs where scalability is crucial, and it achieves this by sampling neighbors and using various aggregation methods.
- **GIN** is highly expressive, capable of distinguishing different graph structures, and is ideal for tasks that require precise structural understanding, such as molecular property prediction.

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

class GraphSAGELayer(nn.Module):
    def __init__(self, in_features, out_features, aggregator="mean"):
        super(GraphSAGELayer, self).__init__()
        # The in_features for the linear layer should be the sum of
        # original in_features and the out_features (due to concatenation)
        self.linear = nn.Linear(in_features + in_features, out_features)
        self.aggregator = aggregator

    def forward(self, x, adj):
        # Aggregate features from neighbors
        if self.aggregator == "mean":
            neighbor_features = torch.matmul(adj, x) / adj.sum(1, keepdim=True)
        elif self.aggregator == "max":
            neighbor_features, _ = torch.max(torch.matmul(adj, x), dim=1)

        # Concatenate self and neighbor features
        out = torch.cat([x, neighbor_features], dim=1)
        return torch.relu(self.linear(out))

# Initialize a GraphSAGE layer with mean aggregation
graphsage_layer = GraphSAGELayer(in_features=2, out_features=4, aggregator="mean")
print("GraphSAGE Layer created.")

GraphSAGE Layer created.


# Demonstration 62: Forward Pass through GraphSAGE Layer
 Perform a forward pass through the GraphSAGE layer.


In [16]:
# Assuming node_features_tensor should be a 2D tensor
node_features_tensor = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=torch.float32)

# Assuming adj_matrix_tensor should be a 2D tensor representing adjacency matrix
adj_matrix_tensor = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]], dtype=torch.float32)

output_graphsage = graphsage_layer(node_features_tensor, adj_matrix_tensor)
print("Output after GraphSAGE layer:\n", output_graphsage)


Output after GraphSAGE layer:
 tensor([[4.3317, 0.3054, 0.0000, 2.8851],
        [2.3624, 0.0000, 0.0000, 0.6322],
        [2.8036, 0.0000, 0.0000, 0.4942]], grad_fn=<ReluBackward0>)


# Demonstration 63: Building a Two-Layer GraphSAGE Model
 We stack two GraphSAGE layers to create a more expressive model for graph representation learning.


In [18]:

class TwoLayerGraphSAGE(nn.Module):
    def __init__(self):
        super(TwoLayerGraphSAGE, self).__init__()
        self.sage1 = GraphSAGELayer(in_features=2, out_features=4, aggregator="mean")
        self.sage2 = GraphSAGELayer(in_features=4, out_features=2, aggregator="mean")

    def forward(self, x, adj):
        x = self.sage1(x, adj)
        x = torch.relu(x)
        x = self.sage2(x, adj)
        return x

# Initialize the two-layer GraphSAGE model
two_layer_graphsage = TwoLayerGraphSAGE()
print("Two-Layer GraphSAGE model created.")


Two-Layer GraphSAGE model created.


# Demonstration 64: Graph Isomorphism Network (GIN) Layer
 GIN is designed to capture graph structure more effectively, using sum aggregation and injective functions.


In [19]:

class GINLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GINLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        # Sum aggregation over neighbors
        aggregated = torch.matmul(adj, x) + x  # Add self-connections (self-loop)
        return torch.relu(self.linear(aggregated))

# Initialize a GIN layer
gin_layer = GINLayer(in_features=2, out_features=4)
print("Graph Isomorphism Network Layer created.")


Graph Isomorphism Network Layer created.


# Demonstration 65: Forward Pass through GIN Layer
 Perform a forward pass through the GIN layer.


In [20]:

output_gin = gin_layer(node_features_tensor, adj_matrix_tensor)
print("Output after GIN layer:\n", output_gin)


Output after GIN layer:
 tensor([[1.4928, 4.2979, 0.3393, 3.2505],
        [0.4514, 2.3641, 0.0498, 1.3902],
        [0.9207, 2.9143, 0.3038, 2.1200]], grad_fn=<ReluBackward0>)


# Demonstration 66: Building a Two-Layer GIN Model
 We can stack multiple GIN layers for a deeper model that captures complex structural information.


In [21]:

class TwoLayerGIN(nn.Module):
    def __init__(self):
        super(TwoLayerGIN, self).__init__()
        self.gin1 = GINLayer(in_features=2, out_features=4)
        self.gin2 = GINLayer(in_features=4, out_features=2)

    def forward(self, x, adj):
        x = self.gin1(x, adj)
        x = torch.relu(x)
        x = self.gin2(x, adj)
        return x

# Initialize the two-layer GIN model
two_layer_gin = TwoLayerGIN()
print("Two-Layer GIN model created.")


Two-Layer GIN model created.


# Demonstration 67: Graph Pooling with GraphSAGE and GIN
 Pooling can be applied to GraphSAGE or GIN models to generate graph-level representations.


In [22]:

# Example of sum pooling after a GraphSAGE model forward pass
output_graphsage = two_layer_graphsage(node_features_tensor, adj_matrix_tensor)
graph_representation_graphsage = output_graphsage.sum(dim=0)
print("Graph-level Representation (GraphSAGE with Sum Pooling):", graph_representation_graphsage)

# Example of mean pooling after a GIN model forward pass
output_gin = two_layer_gin(node_features_tensor, adj_matrix_tensor)
graph_representation_gin = output_gin.mean(dim=0)
print("Graph-level Representation (GIN with Mean Pooling):", graph_representation_gin)


Graph-level Representation (GraphSAGE with Sum Pooling): tensor([0.1770, 1.2386], grad_fn=<SumBackward1>)
Graph-level Representation (GIN with Mean Pooling): tensor([7.5310, 0.0000], grad_fn=<MeanBackward1>)


# Demonstration 68: Graph Classification with GIN Model
 We can use the GIN model for graph classification tasks by applying pooling and fully connected layers.


In [23]:

class GINGraphClassifier(nn.Module):
    def __init__(self):
        super(GINGraphClassifier, self).__init__()
        self.gin1 = GINLayer(in_features=2, out_features=4)
        self.gin2 = GINLayer(in_features=4, out_features=2)
        self.fc = nn.Linear(2, 1)  # Final layer for binary classification

    def forward(self, x, adj):
        x = self.gin1(x, adj)
        x = torch.relu(x)
        x = self.gin2(x, adj)
        x = x.mean(dim=0)  # Mean pooling for graph-level representation
        return self.fc(x)

# Initialize the GIN graph classifier model
gin_classifier = GINGraphClassifier()
print("GIN Graph Classifier model created.")


GIN Graph Classifier model created.


# Demonstration 69: Training GIN Graph Classifier Model
 Train the GIN graph classifier on a simple binary classification task.


In [33]:

# Define label for the graph (e.g., 1 for "positive class")
graph_label = torch.tensor([1.0])

import torch
import torch.nn as nn
import torch.optim as optim

def trainModel(model, node_features, adj_matrix, graph_label, epochs, lr=0.01):
    """
    Trains a graph classification model.

    Args:
        model: The graph classification model (e.g., GINGraphClassifier).
        node_features: Tensor representing node features.
        adj_matrix: Tensor representing the adjacency matrix.
        graph_label: Tensor representing the graph label.
        epochs: Number of training epochs (default: 20).
        lr: Learning rate (default: 0.01).

    Returns:
        Trained model.
    """
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        output = model(node_features, adj_matrix)
        loss = loss_fn(output, graph_label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    return model




# Demonstration 70: Evaluating the GIN Model Predictions
 Evaluate predictions of the GIN model on the graph classification task.


In [35]:
# Example usage:
trained_model = trainModel(gin_classifier, node_features_tensor, adj_matrix_tensor, graph_label,20)

Epoch 1, Loss: 0.376694917678833
Epoch 2, Loss: 0.3735669255256653
Epoch 3, Loss: 0.37046098709106445
Epoch 4, Loss: 0.3673773407936096
Epoch 5, Loss: 0.36431628465652466
Epoch 6, Loss: 0.36127805709838867
Epoch 7, Loss: 0.35826295614242554
Epoch 8, Loss: 0.35527122020721436
Epoch 9, Loss: 0.35230299830436707
Epoch 10, Loss: 0.34935858845710754
Epoch 11, Loss: 0.34643813967704773
Epoch 12, Loss: 0.34354180097579956
Epoch 13, Loss: 0.34066975116729736
Epoch 14, Loss: 0.33782216906547546
Epoch 15, Loss: 0.334999144077301
Epoch 16, Loss: 0.3322007954120636
Epoch 17, Loss: 0.32942721247673035
Epoch 18, Loss: 0.32667848467826843
Epoch 19, Loss: 0.32395467162132263
Epoch 20, Loss: 0.32125580310821533


In [36]:

# Get predictions
output = gin_classifier(node_features_tensor, adj_matrix_tensor).squeeze()
prediction = (torch.sigmoid(output) > 0.5).int()
print("Prediction:", prediction)
print("Actual Label:", graph_label.int())


Prediction: tensor(1, dtype=torch.int32)
Actual Label: tensor([1], dtype=torch.int32)


In [39]:
# Example usage:
trained_model = trainModel(gin_classifier, node_features_tensor, adj_matrix_tensor, graph_label,5)

Epoch 1, Loss: 0.2923385500907898
Epoch 2, Loss: 0.28981316089630127
Epoch 3, Loss: 0.28730711340904236
Epoch 4, Loss: 0.2848205864429474
Epoch 5, Loss: 0.2823539078235626


In [41]:

# Get predictions
output = gin_classifier(node_features_tensor, adj_matrix_tensor).squeeze()
prediction = (torch.sigmoid(output) > 0.5).int()
print("Prediction:", prediction)
print("Actual Label:", graph_label.int())


Prediction: tensor(1, dtype=torch.int32)
Actual Label: tensor([1], dtype=torch.int32)
